Note: in order for this notebook to run, the "updateOutput..." functions must not be using pointers (precedded by &) in the c++ code... SWIG doesn't seem to like it.

In [1]:
from linguamind import linalg as la, nlp, nn

In [2]:
import numpy as np

In [3]:
seed = la.Seed(1)

In [4]:
sample_input = la.Vector(8)

In [5]:
sample_input.uniform(seed)

<linguamind.linalg.Vector; proxy of <Swig Object of type 'Vector *' at 0x109c8e3c0> >

In [6]:
sample_input_np = np.zeros(8)
for i in range(8):
    sample_input_np[i] = sample_input[i]

In [7]:
layer = nn.FlexLinear(8,16)
layer.weights.uniform(seed)

<linguamind.linalg.Matrix; proxy of <Swig Object of type 'Matrix *' at 0x109c8e630> >

In [8]:
weights_np = np.zeros((8,16))
for i in range(8):
    for j in range(16):
        weights_np[i][j] = layer.weights[i][j]

In [9]:
out_np = sample_input_np.dot(weights_np)

In [10]:
delta = la.Vector(16)
delta.uniform(seed)
delta_np = np.array(delta.get())

## Test Dense Dense Forward Prop

In [11]:
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputDenseToDense(sample_input)
    out_lm = layer.getOutput().get()

    for i in range(16):
        assert(out_lm[i] - out_np[i] < 1e-5)
        
    layer.updateInputGrad(delta)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)
        
layer.swapInputOutputSparsity()

layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputDenseToDense(sample_input)
    out_lm = layer.getOutput().get()

    for i in range(16):
        assert(out_lm[i] - out_np[i] < 1e-5)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)

## Test Dense -> Weighted Sparse Forward Prop

In [14]:
layer.getOutput().zero()
layer.getInputGrad().zero()

for iter in range(100):
    layer.updateOutputDenseToWeightedSparse(sample_input,list(range(16)))
    out_lm = layer.getOutput().get()

    for i in range(16):
        assert(np.abs(out_lm[i] - out_np[i]) < 1e-5)
#     assert(out_lm[15] == 0)
    
    layer.updateInputGrad(delta)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)
        
layer.swapInputOutputSparsity()
    
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputDenseToWeightedSparse(sample_input,list(range(16)))
    out_lm = layer.getOutput().get()

    for i in range(16):
        assert(np.abs(out_lm[i] - out_np[i]) < 1e-5)
#     assert(out_lm[15] == 0)
    
    layer.updateInputGrad(delta)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)
        

## Test Weighted Sparse -> Dense Forward Prop

In [18]:
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputWeightedSparseToDense(sample_input,list(range(8)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(out_lm[i] - out_np[i]) < 1e-5)
        
    layer.updateInputGrad(delta)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)

        
layer.swapInputOutputSparsity()
    
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputWeightedSparseToDense(sample_input,list(range(8)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(out_lm[i] - out_np[i]) < 1e-5)

    layer.updateInputGrad(delta)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)


## Test Weighted Sparse -> Weighted Sparse Forward Prop

In [13]:
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputWeightedSparseToWeightedSparse(sample_input,list(range(8)),list(range(16)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(out_lm[i] - out_np[i]) < 1e-5)

        
layer.swapInputOutputSparsity()
    
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputWeightedSparseToWeightedSparse(sample_input,list(range(8)),list(range(16)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(out_lm[i] - out_np[i]) < 1e-5)

## Test Binary Sparse -> Dense Forward Prop

In [20]:
bin_out_np = np.ones(8).dot(weights_np) / 8
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputBinarySparseToDense(list(range(8)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(bin_out_np[i] - out_lm[i]) < 1e-5)

    layer.updateInputGrad(delta)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)

layer.swapInputOutputSparsity()
    
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputBinarySparseToDense(list(range(8)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(bin_out_np[i] - out_lm[i]) < 1e-5)
    
    layer.updateInputGrad(delta)
    
    for i in range(8):
        assert(np.abs(delta_np.dot(weights_np.T)[i] - layer.getInputGrad().get()[i]) < 1e-5)


## Test Binary Sparse -> Weighted Sparse Forward Prop

In [23]:
bin_out_np = np.ones(8).dot(weights_np) / 8
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputBinarySparseToWeightedSparse(list(range(8)),list(range(16)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(bin_out_np[i] - out_lm[i]) < 1e-5)

        
layer.swapInputOutputSparsity()
    
layer.getOutput().zero()
for iter in range(100):
    layer.updateOutputBinarySparseToWeightedSparse(list(range(8)),list(range(16)))
    out_lm = layer.getOutput().get()

    for i in range(8):
        assert(np.abs(bin_out_np[i] - out_lm[i]) < 1e-5)

## Test Matrix Transposition

In [11]:
for iter in range(100):
    mat = la.Matrix(8,16)
    mat.uniform(seed)

    matrix_np = np.zeros((8,16))

    for i in range(8):
        for j in range(16):
            matrix_np[i][j] = mat[i][j]

    mat.transpose()

    for i in range(16):
        for j in range(8):
            assert(matrix_np.T[i][j] - mat[i][j] < 1e-5)